In [21]:
import os
import csv
import json

import pandas as pd
from sklearn.model_selection import train_test_split

SEED = 42

In [22]:
data_path = '../../data/google_translate.jsonl'
with open(data_path, 'r', encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

In [23]:
data[0]

{'message_tree_id': '001199c7-7135-4e64-bee9-48ea862243b4',
 'tree_state': 'ready_for_export',
 'prompt': {'message_id': '001199c7-7135-4e64-bee9-48ea862243b4',
  'user_id': '42398a03-04b3-4028-9d97-1444708e91cd',
  'created_date': '2023-02-04T18:54:01.048716+00:00',
  'text': 'Do you have any information about the Commodore 64?',
  'role': 'prompter',
  'lang': 'en',
  'review_count': 3,
  'review_result': True,
  'deleted': False,
  'synthetic': False,
  'emojis': {'+1': 8, '_skip_reply': 2},
  'replies': [{'message_id': '314ea243-638d-48d6-9f3d-8edf80f85ec2',
    'parent_id': '001199c7-7135-4e64-bee9-48ea862243b4',
    'user_id': '00bbfff8-0e6a-4e46-93cc-65785d78c8bc',
    'created_date': '2023-02-05T00:38:59.927142+00:00',
    'text': "Of course. The Commodore 64 is a 8-bit home computer that was released by Commodore International in August 1982. It is one of the most successful home computers of all time, with over 17 million units sold. It featured 64 kilobytes of RAM, color gra

In [24]:
def assistant(message, history, conversations, split="train"):
    assert message['role'] == 'assistant', 'Message is not from assistant: {}'.format(message)
    for reply in message['replies']:
        conversations = user(reply, history, conversations, split=split)
    return conversations


def user(message, history, conversations, split="train"):
    assert message['role'] == 'prompter', 'Message is not from user: {}'.format(message)
    if 'translation' not in message:
        for reply in message['replies']:
            conversations = assistant(reply, history, conversations, split=split)
    else:
        for reply in message['replies']:
            if 'translation' not in reply:
                conversations = assistant(reply, history, conversations, split=split)
            else:
                rank = reply.get('rank', 3)
                current_message = f'UPORABNIK: {message["translation"]}'
                current_reply = f'ASISTENT: {reply["translation"]}'
                conversations.append((current_message, current_reply, rank, split))
                if history != '':
                    conversations.append((history, current_message, rank, split))
                conversations = assistant(reply, history + current_message + ' ' + current_reply + ' ', conversations, split=split)
    return conversations

def tree_to_conversations(tree, split="train"):
    # get pairs for root prompt and replies
    return user(tree["prompt"], '', [],  split=split)

In [25]:
convos = user(data[1]['prompt'], '', [])

In [26]:
pairs = tree_to_conversations(data[0])

for pair in pairs:
    prompt, reply, rank, split = pair
    print(f"PROMPT: {prompt}")
    print(f"REPLY: {reply}")
    print(f"RANK: {rank}")
    print("====================================="*4)

PROMPT: UPORABNIK: Ali imate kakšne informacije o Commodore 64?
REPLY: ASISTENT: Seveda. Commodore 64 je 8-bitni domači računalnik, ki ga je izdal Commodore International avgusta 1982. Je eden najuspešnejših domačih računalnikov vseh časov, z več kot 17 milijoni prodanih enot. Vseboval je 64 kilobajtov RAM-a, barvno grafiko in zvočni čip SID. Široko se je uporabljal za igre, programiranje in domače aplikacije. Nasledil ga je Commodore 128.

Še vedno obstaja uspešna skupnost navdušencev nad Commodore 64 po vsem svetu, ki še naprej uživajo v igranju in programiranju v sistemu. Več informacij o Commodore 64 najdete na različnih spletnih mestih in forumih, posvečenih sistemu.

Če imate kakršna koli posebna vprašanja o strojni opremi, programski opremi ali programskih jezikih Commodore 64, jih vprašajte.
RANK: 0
PROMPT: UPORABNIK: Ali imate kakšne informacije o Commodore 64?
REPLY: ASISTENT: Seveda, tukaj je kratek povzetek trenutnega članka v Wikipediji:

Commodore 64 (običajne okrajšave: 

### Split the trees in train, validation and test sets

In [27]:
data_train, data_test = train_test_split(data, test_size=0.2, random_state=SEED, shuffle=True)
data_val, data_test = train_test_split(data_test, test_size=0.5, random_state=SEED, shuffle=True)

In [28]:
len(data_train), len(data_val), len(data_test)

(6923, 865, 866)

### Get the pairs from the trees

In [29]:
def generate_pairs(data, split="train"):
    pairs = []
    for tree in data:
        pairs += tree_to_conversations(tree, split)
    df = pd.DataFrame(pairs, columns=["prompt", "reply", "rank", "split"])
    return df

In [30]:
pairs_train = generate_pairs(data_train, "train")
pairs_val = generate_pairs(data_val, "val")
pairs_test = generate_pairs(data_test, "test")

In [31]:
len(pairs_train), len(pairs_val), len(pairs_test)

(54836, 6874, 7140)

In [32]:
pairs_train.head()

,prompt,reply,rank,split
0,UPORABNIK: V mojo hišo bodo prispeli 3 prijate...,ASISTENT: Kaj pa igranje lažnivca? Za to igro ...,0,train
1,UPORABNIK: V mojo hišo bodo prispeli 3 prijate...,ASISTENT: Uno: Je klasična igra za otroke in o...,1,train
2,UPORABNIK: V mojo hišo bodo prispeli 3 prijate...,ASISTENT: Igrate lahko:\n - 8 noro\n - celina\...,2,train
3,UPORABNIK: Napišite hvalnico šolskemu taboru v...,ASISTENT: Spomnim se vas prijatelji\nIn spet s...,0,train
4,UPORABNIK: Ali kaj bolj veselega?,ASISTENT: ♪ Spakirajte in se pridružite zabavi...,0,train


In [33]:
pairs_val

,prompt,reply,rank,split
0,"UPORABNIK: Prosimo, da vljudno zahtevate ponud...",ASISTENT: Dame in gospodje\n\nKmalu se selim i...,0,val
1,UPORABNIK: Napišite mi ustrezno zadevo za e-po...,ASISTENT: Ocena stroškov selitev Nürnberg -> A...,0,val
2,"UPORABNIK: Prosimo, da vljudno zahtevate ponud...",UPORABNIK: Napišite mi ustrezno zadevo za e-po...,0,val
3,UPORABNIK: Katere znamenitosti so v Augsburgu?,ASISTENT: Augsburg je zgodovinsko mesto na Bav...,3,val
4,"UPORABNIK: Prosimo, da vljudno zahtevate ponud...",UPORABNIK: Katere znamenitosti so v Augsburgu?,3,val
...,...,...,...,...
6869,"UPORABNIK: Poiščite besede esdrújula, ki se za...","ASISTENT: To je nekaj besed esdrújula, ki se z...",2,val
6870,UPORABNIK: Odgovor ni pravilen. Besedi enostav...,"ASISTENT: Prav imate, poskušali boste izboljša...",0,val
6871,"UPORABNIK: Poiščite besede esdrújula, ki se za...",UPORABNIK: Odgovor ni pravilen. Besedi enostav...,0,val
6872,UPORABNIK: Odgovor ni pravilen. Besedi enostav...,ASISTENT: Za tako napako se vam iskreno opravi...,1,val


In [34]:
pairs_test

,prompt,reply,rank,split
0,UPORABNIK: Vzemimo naključno matriko $n\krat n...,"ASISTENT: Ocenjevanje natančne verjetnosti, da...",0,test
1,UPORABNIK: Hvala vam! Jasno mi je,ASISTENT: Vedno z veseljem pomagamo! Vas zanim...,0,test
2,UPORABNIK: Vzemimo naključno matriko $n\krat n...,UPORABNIK: Hvala vam! Jasno mi je,0,test
3,UPORABNIK: Hvala vam! Jasno mi je,ASISTENT: prosim!\nV veselje mi je bilo pomaga...,1,test
4,UPORABNIK: Vzemimo naključno matriko $n\krat n...,UPORABNIK: Hvala vam! Jasno mi je,1,test
...,...,...,...,...
7135,"UPORABNIK: Imam elektronski set bobnov, a vsak...",ASISTENT: Pod komplet lahko poskusite postavit...,2,test
7136,UPORABNIK: Kadarkoli poskušam postaviti izolac...,"ASISTENT: Da, obstaja več načinov, kako to pop...",0,test
7137,"UPORABNIK: Imam elektronski set bobnov, a vsak...",UPORABNIK: Kadarkoli poskušam postaviti izolac...,0,test
7138,UPORABNIK: Kadarkoli poskušam postaviti izolac...,"ASISTENT: Težava, da udarec pobegne, je precej...",1,test


In [35]:
# combine all splits to a single dataframe
pairs = pd.concat([pairs_train, pairs_val, pairs_test], ignore_index=True)
pairs.head()

,prompt,reply,rank,split
0,UPORABNIK: V mojo hišo bodo prispeli 3 prijate...,ASISTENT: Kaj pa igranje lažnivca? Za to igro ...,0,train
1,UPORABNIK: V mojo hišo bodo prispeli 3 prijate...,ASISTENT: Uno: Je klasična igra za otroke in o...,1,train
2,UPORABNIK: V mojo hišo bodo prispeli 3 prijate...,ASISTENT: Igrate lahko:\n - 8 noro\n - celina\...,2,train
3,UPORABNIK: Napišite hvalnico šolskemu taboru v...,ASISTENT: Spomnim se vas prijatelji\nIn spet s...,0,train
4,UPORABNIK: Ali kaj bolj veselega?,ASISTENT: ♪ Spakirajte in se pridružite zabavi...,0,train


In [36]:
# drop duplicate rows
pairs = pairs.drop_duplicates(subset=["prompt", "reply"], keep="first")

In [37]:
pairs.to_csv("../../data/processed/context_reply_pairs.csv", index=False, sep=";")

In [38]:
df = pd.read_csv("../../data/processed/context_reply_pairs.csv", sep=";")

In [39]:
from transformers import AutoTokenizer

max_prompt_length = 10  # Maximum allowed length for the prompt

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base", truncation_side ="left")

def tokenize_input(prompt, reply):
    tokenized_prompt = tokenizer.encode(prompt, truncation=True, max_length=max_prompt_length)

    tokenized_reply = tokenizer.encode(reply)
    return tokenized_prompt, tokenized_reply

# Example usage
prompt = "This is a long prompt that needs to be truncated if it exceeds the maximum allowed length. This is a long prompt that needs to be truncated if it exceeds the maximum allowed length."
reply = "This is the reply."

tokenized_input, tokenized_reply = tokenize_input(prompt, reply)

In [40]:
tokenizer.decode(tokenized_input)

'<s> it exceeds the maximum allowed length.</s>'